## Exporting CSVs from the FairNow APIs

#### This generates a CVS export of the Risk and Severity report, containing application, risk, severity, and probability information.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [5]:
import pandas as pd
from utils.fairnow import get_client

client = get_client()

# from getpass import getpass
# import httpx
# from httpx_auth import OAuth2ClientCredentials

# client_id = "{client_id}" # Replace with your Client Id
# client_secret = getpass("Client Secret")
# fairnow_token_endpoint = "https://auth.fairnow.dev/oauth2/token"

# auth = OAuth2ClientCredentials(
#     token_url=fairnow_token_endpoint,
#     client_id=client_id,
#     client_secret=client_secret,
# )

# fairnow_base_url = "https://api.fairnow.dev/v2"
# client = httpx.Client(base_url=fairnow_base_url, auth=auth)

## Applications API call

In [ ]:
application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

## Create Risks DataFrame

In [ ]:
apps_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    risk_items = app.get('risk_item_list', []) or []
    for risk_item in risk_items:
        apps_data.append({
            'application_id': app_id,
            'application_name': app_name,
            'risk_type': risk_item.get('risk_type', ''),
            'risk_type_label': risk_item.get('risk_type_label', ''),
            'severity': risk_item.get('severity', ''),
            'probability': risk_item.get('probability', ''),
        })

# Convert to DataFrame
apps_df = pd.DataFrame(apps_data)
apps_df = apps_df.drop_duplicates()
print(f"Extracted risk items from applications")

## Export to TSV

In [ ]:
if 'apps_df':
    output_file = 'risk_severity_export.tsv'
    apps_df.to_csv(output_file, sep='\t', index=False)
    print(f"\nData exported to {output_file}")
else:
    print("No data to export")